#Libraries and Functions

In [0]:
%restart_python

In [0]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.dates as mdates

#Import Dataset and Feature Engineering

In [0]:
df = spark.table("workspace.timeseries.train2").toPandas()
df_holidays = spark.table("workspace.timeseries.holidays").toPandas()
df_items = spark.table("workspace.timeseries.items").toPandas()
df_oil = spark.table("workspace.timeseries.oil").toPandas()
df_stores = spark.table("workspace.timeseries.stores").toPandas()
df_transactions = spark.table("workspace.timeseries.transactions").toPandas()

In [0]:
#convert date to datetime
df.date = pd.to_datetime(df.date)
df_holidays.date = pd.to_datetime(df_holidays.date)
df_oil.date = pd.to_datetime(df_oil.date)
df_transactions.date = pd.to_datetime(df_transactions.date)

In [0]:
#create a new column with the day of the week
df['week_of_year'] = df['date'].dt.isocalendar().week.astype('int64')

In [0]:
df.head()
#it contains data from Guayas region only
#rows with unit_sales == 0 have been removed: if a product was not sold a specific day, this product has not a row for that day
#the data are between 2013-01-02 and 2014-03-31

In [0]:
df.info()

In [0]:
df_holidays.head(3)

In [0]:
df_items.head(3)
#it contains many families, even though df (i.e. train2) contains products from the 3 most popular families

In [0]:
df_oil.head(3)

In [0]:
df_stores.head(3)

In [0]:
df_transactions.head(3)

#train2 and item datasets merging

In [0]:
df_items.head(3)

In [0]:
df.head(3)

In [0]:
df_train_items = pd.merge(df, df_items, on = 'item_nbr', how='left')
df_train_items.head(5)

#EDA

##Font size variables

In [0]:
# Font size variables for plot text elements
title_fontsize = 16+5
label_fontsize = 12+5
tick_fontsize = 10+5
legend_fontsize = 12+5

##Top 3 Item Families

In [0]:
# Plotting the number of items per family (for top 3 families only)
# Compute the number of items per family and select the top 3 families
items_per_family = df_items['family'].value_counts().reset_index()
items_per_family.columns = ['Family', 'Item Count']
top_3_families = items_per_family.head(3)  # Keep only the top 3 families

# Plot the top 3 families
plt.figure(figsize=(10, 4))
sns.barplot(data=top_3_families, x='Family', y='Item Count')
plt.title("Top 3 Families by Number of Items", fontsize=title_fontsize)
plt.xlabel("Family", fontsize=label_fontsize)
plt.ylabel("Number of Items", fontsize=label_fontsize)
plt.xticks(fontsize=tick_fontsize)
plt.yticks(fontsize=tick_fontsize)

# Save the plot as a PNG file
plt.savefig("/Workspace/Users/filippopedrini95@gmail.com/TimeSeriesProject/MS_DS_TimeSeriesCourse_Project/Visualizations/top3_families.png", dpi=300, bbox_inches='tight')

plt.show()

##Sales Trend Overtime

In [0]:
df_train_items.head(3)

In [0]:
df_train_items.family.value_counts()

In [0]:
sales_by_date = df.groupby('date')['unit_sales'].sum().asfreq('D').fillna(0)

grocery_sales_by_date = df_train_items[df_train_items.family == 'GROCERY I'].groupby('date')['unit_sales'].sum().asfreq('D').fillna(0)
cleaning_sales_by_date = df_train_items[df_train_items.family == 'CLEANING'].groupby('date')['unit_sales'].sum().asfreq('D').fillna(0)
beverages_sales_by_date = df_train_items[df_train_items.family == 'BEVERAGES'].groupby('date')['unit_sales'].sum().asfreq('D').fillna(0)

In [0]:
series_list = [
    (sales_by_date, 'Total Sales', 'C0'),
    (grocery_sales_by_date, 'Grocery Sales', 'C1'),
    (cleaning_sales_by_date, 'Cleaning Sales', 'C2'),
    (beverages_sales_by_date, 'Beverages Sales', 'C3')
]

for idx, (series, title, color) in enumerate(series_list):
    plt.figure(figsize=(20, 5))
    plt.plot(series.index, series.values, color=color)
    plt.title(title, fontsize=title_fontsize)
    plt.ylabel('Unit Sales', fontsize=label_fontsize)
    if idx == 3:
        plt.xlabel('Date', fontsize=label_fontsize)
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    plt.xticks(rotation=45, fontsize=tick_fontsize)
    plt.yticks(fontsize=tick_fontsize)
    plt.tight_layout()
    plt.show()

In [0]:
plt.figure(figsize=(20,5))
plt.plot(sales_by_date.index, sales_by_date.values, label='Total Sales')
plt.plot(grocery_sales_by_date.index, grocery_sales_by_date.values, label='Grocery Sales')
plt.plot(cleaning_sales_by_date.index, cleaning_sales_by_date.values, label='Cleaning Sales')
plt.plot(beverages_sales_by_date.index, beverages_sales_by_date.values, label='Beverages Sales')
plt.xlabel("Date", fontsize=label_fontsize)
plt.ylabel("Unit Sales", fontsize=label_fontsize)
plt.title("Sales Over Time by Family", fontsize=title_fontsize)
plt.xticks(fontsize=tick_fontsize)
plt.yticks(fontsize=tick_fontsize)
plt.legend(fontsize=legend_fontsize)
plt.show()

In [0]:
plt.figure(figsize=(20,8))
plt.plot(sales_by_date.index, sales_by_date.values, label='Total Sales')

plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.xticks(rotation=45, fontsize=tick_fontsize)
plt.yticks(fontsize=tick_fontsize)

plt.xlabel("", fontsize=label_fontsize)
plt.ylabel("unit sales", fontsize=label_fontsize)
plt.title("Total Sales Over Time", fontsize=title_fontsize, fontweight='bold')

# Add a thin, black, dashed vertical line at 31 December 2013
#plt.axvline(pd.Timestamp('2014-01-01'), color='black', linestyle='--', linewidth=0.7)

# Save the plot as a PNG file
plt.savefig("/Workspace/Users/filippopedrini95@gmail.com/TimeSeriesProject/MS_DS_TimeSeriesCourse_Project/Visualizations/sales_trend_overtime.png", dpi=300, bbox_inches='tight')

plt.show()

##Weekly Sales Heatmap

In [0]:
adj_size = 0
# Aggregating sales by year and week
sales_by_week = df.groupby(['year', 'week_of_year'])['unit_sales'].sum().unstack()
sales_by_week.loc[2013, 1] = np.nan #I remove first week of 2013 because incomplete
sales_by_week.loc[2014, 14] = np.nan #I remove last week of the dataset because incomplete

plt.figure(figsize=(20, 3))  # Increase figure size for better visibility
ax = sns.heatmap(
    sales_by_week,
    cmap='coolwarm',  # Use a diverging colormap for better contrast
    linewidths=0.5,  # Add lines between cells for clarity
    linecolor='white',  # Use white lines for a cleaner look
    cbar_kws={'label': 'Sales Volume'}  # Add a descriptive colorbar label
)

plt.title('Weekly Sales Trends Over Years', fontsize=title_fontsize+adj_size, fontweight='bold')
plt.xlabel('Calendar Week', fontsize=label_fontsize+adj_size)
plt.ylabel('', fontsize=label_fontsize+adj_size)

# Show only every other tick label on X axis
xticklabels = ax.get_xticklabels()
for i, label in enumerate(xticklabels):
    if i % 2 != 0:
        label.set_visible(False)
ax.set_xticklabels(xticklabels, fontsize=tick_fontsize+adj_size)

ax.set_yticklabels(ax.get_yticklabels(), fontsize=tick_fontsize+adj_size)

cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=legend_fontsize-7)
cbar.set_label('Sales Volume', fontsize=legend_fontsize-7)

# Save the plot as a PNG file
plt.savefig("/Workspace/Users/filippopedrini95@gmail.com/TimeSeriesProject/MS_DS_TimeSeriesCourse_Project/Visualizations/WeeklySales_heatmap.png", dpi=300, bbox_inches='tight')

plt.show()

##Holiday Impact Analysis

In [0]:
# Select holidays for Guayas region (Guayaquil or Ecuador) within the specified date range
guayas_holidays = df_holidays[
    ((df_holidays.locale_name == "Guayaquil") | (df_holidays.locale_name == "Ecuador"))  # Filter for relevant locales
    & ((df_holidays.date > '2013-01-01') & (df_holidays.date <= '2014-03-31'))          # Filter for date range
].copy()

# Remove character '-', '+' and any numbers from the holiday description
guayas_holidays.loc[:, 'description'] = guayas_holidays['description'].str.replace(r'[-+]?\d+', '', regex=True)

# Create a 'is_holiday' column to indicate whether a date is a holiday
guayas_holidays.loc[:, 'is_holiday'] = True

guayas_holidays

In [0]:
# create a dataframe containing sales grouped by date
sales_by_date_df = df.groupby('date')['unit_sales'].sum().asfreq('D').reset_index()

# merge the sales dataframe with some features of the holidays dataframe
sales_by_date_df = pd.merge(sales_by_date_df, guayas_holidays[['date', 'is_holiday', 'description']], left_on='date', right_on='date', how='left')

# fill missing values for 'is_holiday' with False
sales_by_date_df['is_holiday'] = sales_by_date_df['is_holiday'].fillna(False).astype(bool)
sales_by_date_df.head(12)

In [0]:
# Create a "day_type" column, assigning 'weekday' or 'weekend' to each date based on the day of the week
sales_by_date_df['day_type'] = sales_by_date_df.date.dt.dayofweek.replace({0: 'weekday', 1: 'weekday', 2: 'weekday', 3: 'weekday', 4: 'weekday', 5: 'weekend', 6: 'weekend'})

# Overwrite "day_type" as "holiday" for dates that are holidays
sales_by_date_df.loc[sales_by_date_df.is_holiday, 'day_type'] = 'holiday'

# Fill missing holiday descriptions with the corresponding day type
sales_by_date_df['description'] = sales_by_date_df['description'].fillna(sales_by_date_df['day_type'])

sales_by_date_df.head(12)

In [0]:
# This code visualizes the average unit sales by day type (weekday, weekend, holiday) and by holiday description.
# It creates two bar plots: one showing average sales for each day type, and another breaking down average sales by specific holiday or day type.

adj_size = 5
suptitle_fontsize = 20 + adj_size

fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(20,16))

fig.suptitle("Average Unit Sales by Day Type and Holiday Type", fontsize=suptitle_fontsize, fontweight='bold', y=1)

pastel_green = "#4CAF50" 
pastel_blue = "C0"  

# average unit_sales by day_type
day_type_data = sales_by_date_df.groupby('day_type')['unit_sales'].mean().reset_index()
day_type_data['color'] = [pastel_green if dt in ['weekday', 'weekend'] else pastel_blue for dt in day_type_data['day_type']]
sns.barplot(
    data=day_type_data, 
    y='unit_sales', 
    x='day_type', 
    ax=ax[0], 
    hue='day_type', 
    palette=dict(zip(day_type_data['day_type'], day_type_data['color'])), 
    dodge=False, 
    legend=False
)
ax[0].set_title("Details by Day Type", fontsize=title_fontsize + adj_size)
ax[0].set_xlabel("", fontsize=label_fontsize + adj_size)
ax[0].set_ylabel("average unit sales", fontsize=label_fontsize + adj_size)
ax[0].tick_params(axis='x', labelsize=tick_fontsize + adj_size)
ax[0].tick_params(axis='y', labelsize=tick_fontsize + adj_size)

# average unit_sales by description, i.e. by holiday name and/or day type
desc_data = sales_by_date_df.groupby('description')['unit_sales'].mean().reset_index().sort_values(by='unit_sales', ascending=False)
desc_data['color'] = [pastel_green if desc in ['weekday', 'weekend'] else pastel_blue for desc in desc_data['description']]
sns.barplot(
    data=desc_data, 
    y='unit_sales', 
    x='description', 
    ax=ax[1], 
    hue='description', 
    palette=dict(zip(desc_data['description'], desc_data['color'])), 
    dodge=False, 
    legend=False
)
ax[1].set_title("Breakdown by Holiday Type", fontsize=title_fontsize + adj_size)
ax[1].set_xlabel("", fontsize=label_fontsize + adj_size)
ax[1].set_ylabel("average unit sales", fontsize=label_fontsize + adj_size)
plt.setp(ax[1].get_xticklabels(), rotation=45, ha='right', fontsize=tick_fontsize + adj_size)
plt.setp(ax[1].get_yticklabels(), fontsize=tick_fontsize + adj_size)

plt.tight_layout()
plt.subplots_adjust(hspace=0.2)

# Save the plot as a PNG file
plt.savefig("/Workspace/Users/filippopedrini95@gmail.com/TimeSeriesProject/MS_DS_TimeSeriesCourse_Project/Visualizations/holida_impact_analysis.png", dpi=300, bbox_inches='tight')

plt.show()

##Perishable vs. non-perishable sales comparison

#Saving Data

In [0]:
#spark.sql("DROP TABLE IF EXISTS workspace.timeseries.train2")
#spark.sql(f"DROP TABLE IF EXISTS workspace.timeseries.holidays")
#spark.sql(f"DROP TABLE IF EXISTS workspace.timeseries.items")
#spark.sql(f"DROP TABLE IF EXISTS workspace.timeseries.oil")
#spark.sql(f"DROP TABLE IF EXISTS workspace.timeseries.stores")
#spark.sql(f"DROP TABLE IF EXISTS workspace.timeseries.transactions")

In [0]:
#spark_df = spark.createDataFrame(df)
#spark_df.write.format("delta").mode("overwrite").saveAsTable("workspace.timeseries.train2")

#spark_df = spark.createDataFrame(df_holidays)
#spark_df.write.format("delta").mode("overwrite").saveAsTable("workspace.timeseries.holidays")

#spark_df = spark.createDataFrame(df_items)
#spark_df.write.format("delta").mode("overwrite").saveAsTable("workspace.timeseries.items")

#spark_df = spark.createDataFrame(df_oil)
#spark_df.write.format("delta").mode("overwrite").saveAsTable("workspace.timeseries.oil")

#spark_df = spark.createDataFrame(df_stores)
#spark_df.write.format("delta").mode("overwrite").saveAsTable("workspace.timeseries.stores")

#spark_df = spark.createDataFrame(df_transactions)
#spark_df.write.format("delta").mode("overwrite").saveAsTable("workspace.timeseries.transactions")